In [ ]:
%%bash
pip install transformers
pip install sentencepiece
pip install accelerate

In [3]:
import torch
from transformers import pipeline
import os

In [ ]:
# load model
dolly_pipeline = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto")

In [ ]:
import os
from google.colab import userdata

# Access the secret
hf_token = userdata.get('HF_TOKEN')

# Set the environment variable for Hugging Face
os.environ["HF_TOKEN"] = hf_token

In [7]:
from IPython.display import Markdown, display

def show_py_file(filepath):
    """Display Python file contents as markdown code block"""
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            content = file.read()

        # Create markdown with python syntax highlighting
        markdown_content = f"```python\n{content}\n```"
        display(Markdown(markdown_content))
    except FileNotFoundError:
        print(f"File not found: {filepath}")
    except Exception as e:
        print(f"Error reading file: {e}")

from IPython.display import Markdown, display
def in_md(md_txt):
    md_formated_txt = f"--- Response -------<br/>{md_txt}<br/>-------------------"
    display(Markdown(md_formated_txt))

Chat with Dolly

In [11]:
def chat(prompt):
  response = dolly_pipeline(prompt)
  in_md(response[0]['generated_text'])



In [12]:
prompt = "What is the meaning of life?"
chat(prompt)

--- Response -------<br/>The meaning of life is to live a fulfilled life.<br/>-------------------